In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# Child and Adult Care Food Program (CACFP) Sites
`cde.ca.gov-cacfp_sites-2` Last Update: 2019-06-18T04:32:45

_The Child and Adult Care Food Program (CACFP) provides federal funding to participating child care and adult day care centers, allowing the centers to serve nutritious meals to enrolled participants_


California Department of Education CACFP sites file, with extra features,
include:

* Sites geocoded to Lat/Lon and Census tract
* The `program_types` field is broken out into dummy variables
* CDS codes are matched from the CDE schools list. 

## CDS Code Matching

There are 5 different methods for matching sites to schools in the schools
file, each with its own code:

* `fhs`: Fuzzy hash string, a has string based n the address. 
* `fhs_nz`: The fuzzy hash strin, but without the zip code, which occasionally is wrong or changes. 
* `name_city`: A lightly normalized ( alpha only, lowercase, combination of the site name and the site city
* `name_county`: A lightly normalized combination of the site name and county
* `mp_name`: The site name and county, with each word converted to it's metaphone. 

The matching method code is included in the `match_type` columns. For many of
the matching types there can be more than one CDS code associated with the
matching string, particularly the strings based on addresses. The ``match_qc``
value is the inverse of the number of values with the same matching code, so if
the algorithm selected the first of 5 CDS codes, the ``match_qc`` value is .2

Only about of the 35% of the 5076 sites are matched. The number matched for each type of matching tecchnique is: 

    Not Matched    3292
    fhs            1602
    name_city       161
    name_county      19
    mp_name           1
    fhs_nz            1
    
## Versions

1. Initial version
2. Add faked CDS codes for sites that are not schools
## Documentation Links

* [Program home page](https://www.cde.ca.gov/ls/nu/cc/index.asp) 
* [Download page](https://www.cde.ca.gov/ds/sh/sn/cacfpcentermap.asp) 

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [cacfp_sites](notebooks/ProcessCACFP.ipynb#cacfp)**. CACFP sites, with location geocode and program type dummy variables

## References

* **[site_geocodes](data/geocodes.csv)**. Externally geocoded site locations. Geocoded with Geocodio.io
* **[cacfp_sites_source](https://www.cde.ca.gov/ds/sh/sn/documents/apcacfpcentersites2019.xlsx#&headers=4&start=5)**. List of  Child and Adult Care Food Program (CACFP) sites
* **[schools](metapack+http://library.metatab.org/cde.ca.gov-schools-1.csv#public_schools)**. California schools and districts
* **[ca_county_codes](metapack+http://library.metatab.org/sandiegodata.org-county_codes-ca-1.csv#ca_county_codes)**. California government codes for counties, used in  CDS codes

In [3]:
gc = pkg.reference('site_geocodes').dataframe()
gc.head().T

,0,1,2,3,4
id,0,1,2,3,4
address,4700 EAST 14TH STREET,3500 EAST 9TH ST,670 WEST MIDWAY,1901 THIRD STREET,555 RALPH M. APPEZATO WAY
state,CA,CA,CA,CA,CA
city,OAKLAND,OAKLAND,ALAMEDA,ALAMEDA,ALAMEDA
zip,94601,94601,94501,94501,94501
Latitude,37.7706,37.7727,37.7852,37.7787,37.7706
Longitude,-122.21,-122.225,-122.299,-122.288,-122.265
Accuracy Score,1,1,1,1,0.33
Accuracy Type,rooftop,rooftop,rooftop,rooftop,place
Number,4700,3500,670,1901,NaN


In [4]:
df = pkg.reference('cacfp_sites_source').dataframe()

df.columns = """site_county
sponsor_name
site_name
site_address
site_city
site_state
site_zip
site_contact
site_contact_phone_number
site_program_type""".splitlines()
df.head()

,site_county,sponsor_name,site_name,site_address,site_city,site_state,site_zip,site_contact,site_contact_phone_number,site_program_type
0,ALAMEDA,24-HR OAKLAND PARENT-TEACHER CHILDRENS CENTER,24 HOUR OAKLAND DAY CARE CTR,4700 EAST 14TH STREET,OAKLAND,CA,94601,MS. GWEN BELL-BABAOYE,510-532-0574,PRESCHOOL AGE CENTER (CCC)
1,ALAMEDA,24-HR OAKLAND PARENT-TEACHER CHILDRENS CENTER,24 HOUR OAKLAND DAY CARE CTR,3500 EAST 9TH ST,OAKLAND,CA,94601,MS. DELORES SANDERS,510-261-0162,PRESCHOOL AGE CENTER (CCC)
2,ALAMEDA,ALAMEDA FAMILY SERVICES,ALAMEDA HEAD START - SUE MATHESON INFANT CENTER,670 WEST MIDWAY,ALAMEDA,CA,94501,MS. DONNA ADAMS,510-521-2891,"PRESCHOOL AGE CENTER (CCC), INFANT/TODDLER (ITC)"
3,ALAMEDA,ALAMEDA FAMILY SERVICES,ALAMEDA HS/AGUILAR,1901 THIRD STREET,ALAMEDA,CA,94501,MS. BING ZHANG,510-865-6304,HEAD START CENTER (HSC)
4,ALAMEDA,ALAMEDA FAMILY SERVICES,ALAMEDA HS/EHS-COLLEGE OF ALAMEDA,555 RALPH M. APPEZATO WAY,ALAMEDA,CA,94501,MS. CONNIE MA,510-748-9618,HEAD START CENTER (HSC)


In [5]:
df.describe().T

,count,unique,top,freq
site_county,5076,56,LOS ANGELES,1552
sponsor_name,5076,1184,FRESNO UNIFIED SCHOOL DISTRICT,89
site_name,5076,4950,WASHINGTON ELEMENTARY,7
site_address,5076,5023,490 EL CAMINO REAL,4
site_city,5076,686,LOS ANGELES,484
site_state,5076,1,CA,5076
site_zip,5076,1137,90011,33
site_contact,5076,4167,MRS. SARA MARAGNI,31
site_contact_phone_number,5076,4156,916-566-1600,41
site_program_type,5076,55,AT RISK SITE (ATR),1205


In [6]:
pt_map = {
    'ADULT DAY CARE \(ADC\)' : 'adc',
    'ADULT DAY HEALTH \(ADH\)': 'adh',
    'AT RISK SITE \(ATR\)': 'atr',
    'EMERGENCY SHELTER \(HOM\)': 'hom',
    'HEAD START CENTER \(HSC\)': 'hsc',
    'INFANT/TODDLER \(ITC\)': 'itc',
    'OTHER': 'other',
    'PRESCHOOL AGE CENTER \(CCC\)': 'ccc',
    'SCHOOL AGE CENTER \(SAC\)': 'sac',
    'STATE PRE SCHOOL CENTER \(SPS\)': 'sps'
}

# Create the columns and set them to False. Then select just the rows that contain
# one of the strings, and set all of those to True
for k, v in pt_map.items():
    df[v] = False
    df.loc[df.site_program_type.str.contains(k),[v]]  = True

In [7]:
df.head().T

,0,1,2,3,4
site_county,ALAMEDA,ALAMEDA,ALAMEDA,ALAMEDA,ALAMEDA
sponsor_name,24-HR OAKLAND PARENT-TEACHER CHILDRENS CENTER,24-HR OAKLAND PARENT-TEACHER CHILDRENS CENTER,ALAMEDA FAMILY SERVICES,ALAMEDA FAMILY SERVICES,ALAMEDA FAMILY SERVICES
site_name,24 HOUR OAKLAND DAY CARE CTR,24 HOUR OAKLAND DAY CARE CTR,ALAMEDA HEAD START - SUE MATHESON INFANT CENTER,ALAMEDA HS/AGUILAR,ALAMEDA HS/EHS-COLLEGE OF ALAMEDA
site_address,4700 EAST 14TH STREET,3500 EAST 9TH ST,670 WEST MIDWAY,1901 THIRD STREET,555 RALPH M. APPEZATO WAY
site_city,OAKLAND,OAKLAND,ALAMEDA,ALAMEDA,ALAMEDA
site_state,CA,CA,CA,CA,CA
site_zip,94601,94601,94501,94501,94501
site_contact,MS. GWEN BELL-BABAOYE,MS. DELORES SANDERS,MS. DONNA ADAMS,MS. BING ZHANG,MS. CONNIE MA
site_contact_phone_number,510-532-0574,510-261-0162,510-521-2891,510-865-6304,510-748-9618
site_program_type,PRESCHOOL AGE CENTER (CCC),PRESCHOOL AGE CENTER (CCC),"PRESCHOOL AGE CENTER (CCC), INFANT/TODDLER (ITC)",HEAD START CENTER (HSC),HEAD START CENTER (HSC)


In [8]:
## Extract addresses from the pre-compiled geocode, 
# or if not there, call the census geocoder. 
import shelve
from  tqdm import tqdm_notebook
import censusgeocode as cg

def gcr_from_gc(r):
    coors = r.get('response')[0].get('coordinates')
    ct = r.get('response')[0].get('geographies').get('Census Tracts')[0]

    return dict(
        lat = coors['y'],
        lon = coors['x'],
        geoid = ct['geoid']
    )
  
def mk_key_gc(row):
    args = dict(street=row.address, city=row.city, state=row.state, zipcode=row.zip)
    return '-'.join(list(args.values()))
  
def mk_key_df(row):
    args = dict(street=row.site_address, city=row.site_city, state=row.site_state, zipcode=row.site_zip)
    return '-'.join(list(args.values()))
    
extant_gc = {}
for i,(idx, row) in enumerate(gc.iterrows()):
    
    key = mk_key_gc(row)
    
    try:
        geoid = '14000US'+str(str(row['County FIPS']).zfill(5)+str(int(row['Census Tract Code'])).zfill(6))
    except ValueError:
        geoid = None
    
    gcr = dict(
            lat = row.Latitude,
            lon = row.Longitude,
            geoid=geoid,
            address_key=key)

    extant_gc[key] = gcr  
    
df['address_key'] = df.apply(mk_key_df, axis = 1) 
    
dfx = pd.DataFrame(extant_gc.values())

dfx.head()
    

,address_key,geoid,lat,lon
0,4700 EAST 14TH STREET-OAKLAND-CA-94601,14000US06001407400,37.770622,-122.210436
1,3500 EAST 9TH ST-OAKLAND-CA-94601,14000US06001406100,37.772727,-122.225031
2,670 WEST MIDWAY-ALAMEDA-CA-94501,14000US06001428700,37.785245,-122.299103
3,1901 THIRD STREET-ALAMEDA-CA-94501,14000US06001427600,37.778734,-122.288263
4,555 RALPH M. APPEZATO WAY-ALAMEDA-CA-94501,14000US06001427800,37.770563,-122.264779


In [9]:
merged = df.merge(dfx, on='address_key').drop(columns='address_key')

merged = merged[[
'site_name',
'site_county',
'site_address',
'site_city',
'site_state',
'site_zip',
'geoid',
'lat',
'lon',
'sponsor_name',
'site_contact',
'site_contact_phone_number',
'site_program_type',
'adc',
'adh',
'atr',
'hom',
'hsc',
'itc',
'other',
'ccc',
'sac',
'sps'
]]

merged = merged.replace({True:1, False:0})


In [10]:
# Create the file to send to the batch geocoder, geocodio.io
# unique id, street address, state, city, zip code
df[['site_address','site_state','site_city','site_zip']].to_csv('gc_batch.csv', header=False)

Now we can try to match up the CDS codes

In [11]:

import sys
!{sys.executable} -mpip install --quiet textdistance address_parser py_stringmatching nltk phonetics

from address_parser import Parser
from py_stringmatching import Levenshtein

import re
from py_stringmatching import AlphabeticTokenizer
from phonetics import metaphone
al_tok = AlphabeticTokenizer()

parser = Parser() 

def ps_hash(row):  
    p = parser.parse(row.street, city=row.city, state = row.state, zip=row.zip)
    return p.hash.fuzzy_hash_string

def df_hash(row):  
    try:
        p = parser.parse(row.address, city=row.city, state = row.state, zip=row.zip)
        return p.hash.fuzzy_hash_string
    except IndexError:
        return None
    
ps = pkg.reference('schools').dataframe()

#pkg = mp.jupyter.open_package()
cacfp = merged #pkg.resource('cacfp_sites').dataframe()
cacfp.rename(columns={ e:e.replace('site_','') for e in ['site_name','site_county','site_address','site_city','site_state','site_zip']}, inplace=True)

def mp_name(v):
    """Metaphone-ised name, with the county"""
    return ' '.join(sorted(metaphone(w) for w in al_tok.tokenize(row['school'].lower()+' '+row.county.lower())))

def df_mp_name(row):
    """Metaphone-ised name, with the county"""
    return ' '.join(sorted(metaphone(w) for w in al_tok.tokenize(row['name'].lower()+' '+row.county.lower())))


def clean(*a):

    return ','.join(re.sub(r'[^\w\s]','',s.strip().lower()) for s in a )

cacfp['name'] = cacfp.name.apply(lambda v: v.strip())
cacfp['fhs'] = cacfp.apply(df_hash,axis=1)
cacfp['fhs_nz'] = cacfp.fhs.apply(lambda v: '|'.join(v.split('|')[-1]) if v else None)
cacfp['match_name'] = cacfp.apply(lambda r: clean(r['name'],r.city),axis=1)
cacfp['match_name_county'] = cacfp.apply(lambda r: clean(r['name'],r.county),axis=1)
cacfp['mp_name'] = cacfp.apply(lambda r: df_mp_name(r),axis=1)

In [12]:
from collections import defaultdict

fhs_map = defaultdict(list)
fhs_nz_map = defaultdict(list)
match_name_city_map = defaultdict(list)
match_name_county_map = defaultdict(list)
mp_name_map = defaultdict(list)

parser = Parser() 

def strip_zip(v):
    '|'.join(v.split('|')[-1])  

fhs_ignores = [
    'none|.|.|ntt|ntt|no data', # No address, b/c its closed
    '50|.|mrlnt|smfl|k|93065', # Physical site for a virtual charter that operates in many districts
    '4684|.|antr|antr|k|91764', # Another alternative charter
    '1398|.|sprpr|alsntr|k|92243', # Imperial county office of ed
]

fhs_nz_ignores = [ strip_zip(v) for v in fhs_ignores ]

for idx, row in ps.iterrows():
    
    p = parser.parse(row.street, city=row.city, state = row.state, zip=row.zip)
    fhs = p.hash.fuzzy_hash_string

    if fhs not in fhs_ignores:
        fhs_map[fhs].append(row.cdscode )

    # Fuzzy Hash string, without the final zip code
    fhs_nz = strip_zip(fhs)
    
    if fhs not in fhs_nz_ignores:
        fhs_nz_map[fhs_nz].append(row.cdscode )
        
    match_name_city_map[clean(row.school,row.city)].append(row.cdscode)
    
    match_name_county_map[clean(row.school,row.county)].append(row.cdscode)
    
    mp_name_map[mp_name(row.school)].append(row.cdscode)
    

In [13]:

cacfp['cdscode'] = None
cacfp['match_type'] = None
cacfp['match_qc'] = None

fhs_map = dict(fhs_map.items())
fhs_nz_map = dict(fhs_nz_map.items())
match_name_city_map = dict(match_name_city_map.items())
match_name_county_map = dict(match_name_county_map.items())
mp_name_map = dict(mp_name_map.items())

for idx, row in cacfp.iterrows():
    
    if row.fhs in fhs_map:
        cacfp.loc[idx,'cdscode'] = fhs_map[row.fhs][0]
        cacfp.loc[idx,'match_type'] = 'fhs'
        cacfp.loc[idx,'match_qc'] = 1. / len(fhs_map[row.fhs])
        continue
        
    if row.match_name in match_name_city_map:
        cacfp.loc[idx,'cdscode'] = match_name_city_map[row.match_name][0]
        cacfp.loc[idx,'match_type'] = 'name_city'
        cacfp.loc[idx,'match_qc'] = 1. / len(match_name_city_map[row.match_name])
        continue
        
    if row.match_name_county in match_name_county_map:
        cacfp.loc[idx,'cdscode'] = match_name_county_map[row.match_name_county][0]
        cacfp.loc[idx,'match_type'] = 'name_county'
        cacfp.loc[idx,'match_qc'] = 1. / len(match_name_county_map[row.match_name_county]) 
        continue
        
    if row.fhs_nz in fhs_nz_map:
        cacfp.loc[idx,'cdscode'] = fhs_nz_map[row.fhs_nz][0]
        cacfp.loc[idx,'match_type'] = 'fhs_nz'
        cacfp.loc[idx,'match_qc'] = 1. / len(fhs_nz_map[row.fhs_nz])
        continue
        
    if row.mp_name in mp_name_map:
        cacfp.loc[idx,'cdscode'] = mp_name_map[row.mp_name][0]
        cacfp.loc[idx,'match_type'] = 'mp_name'
        cacfp.loc[idx,'match_qc'] = 1. / len(mp_name_map[row.mp_name])
        continue
        
#unmatched = cacfp[cacfp.match_type.isnull()]   
#unmatched.to_csv('unmatched.csv')
cacfp.drop(columns=['fhs', 'fhs_nz', 'match_name', 'match_name_county', 'mp_name'], inplace=True)
cacfp.match_type.value_counts(dropna=False)

NaN            3292
fhs            1602
name_city       161
name_county      19
mp_name           1
fhs_nz            1
Name: match_type, dtype: int64

In [14]:
# They are supposed to be strings, with a leading zero. 
cacfp['cdscode'] = cacfp.cdscode.apply(lambda v: str(v).zfill(14) if v else None)

# Create new CDS codes for the sites with missing codes. 

In [15]:
cc = pkg.reference('ca_county_codes').dataframe()
cc['county'] = cc.county.apply(lambda v: v.upper())
cc['code'] = cc.code.astype(str)
cc['code'] = cc.code.apply(lambda v: v.zfill(2))

cc_map = dict(zip(cc.county, cc.code))

In [16]:
# CDS CODES
# 2 for county, although often the leading zero is removed
# 5 for the district
# 7 for the school
no_code = cacfp[cacfp.cdscode.isnull()].copy()
no_code['county_code'] = cacfp.county.apply(lambda v: cc_map.get(v))
no_code['in_county_id'] = no_code.groupby('county').cumcount()+1
no_code['cdscode'] = no_code.apply(lambda r: r.county_code + 'CACFP' + str(r.in_county_id).zfill(7), axis=1)
cacfp.loc[no_code.index,'cdscode'] = no_code.cdscode
cacfp.head().T

,0,1,2,3,4
name,24 HOUR OAKLAND DAY CARE CTR,24 HOUR OAKLAND DAY CARE CTR,ALAMEDA HEAD START - SUE MATHESON INFANT CENTER,ALAMEDA HS/AGUILAR,ALAMEDA HS/EHS-COLLEGE OF ALAMEDA
county,ALAMEDA,ALAMEDA,ALAMEDA,ALAMEDA,ALAMEDA
address,4700 EAST 14TH STREET,3500 EAST 9TH ST,670 WEST MIDWAY,1901 THIRD STREET,555 RALPH M. APPEZATO WAY
city,OAKLAND,OAKLAND,ALAMEDA,ALAMEDA,ALAMEDA
state,CA,CA,CA,CA,CA
zip,94601,94601,94501,94501,94501
geoid,14000US06001407400,14000US06001406100,14000US06001428700,14000US06001427600,14000US06001427800
lat,37.7706,37.7727,37.7852,37.7787,37.7706
lon,-122.21,-122.225,-122.299,-122.288,-122.265
sponsor_name,24-HR OAKLAND PARENT-TEACHER CHILDRENS CENTER,24-HR OAKLAND PARENT-TEACHER CHILDRENS CENTER,ALAMEDA FAMILY SERVICES,ALAMEDA FAMILY SERVICES,ALAMEDA FAMILY SERVICES
